## Introduction

[Video Walkthrough](https://www.youtube.com/watch?v=Obbn15rZfvQ&list=PLypX5sYuDqvrqsXTw876gGHosCKvK_7QS&index=13)

This notebook demonstrates the implementation of a Retrieval-Augmented Generation (RAG) pipeline using KDB.AI and Large Language Models. By the end of this tutorial, you'll understand how to leverage vector databases and LLMs to create an effective RAG system.

### Setup and Dependencies
Install kdbai_client and import the necessary dependencies

##### Install Required Libraries

In [ ]:
# Install required libraries
!pip install llama-index fastembed openai kdbai_client onnxruntime==1.19.2

##### Import Dependencies

In [2]:
import os
from getpass import getpass
import kdbai_client as kdbai
import time
from llama_index.core import Document, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
from fastembed import TextEmbedding
import openai
import textwrap

##### Connect to KDB.AI

In [3]:
# Set up OpenAI API key
OPENAI_API_KEY = (
    os.environ["OPENAI_API_KEY"]
    if "OPENAI_API_KEY" in os.environ
    else getpass("OpenAI API key: ")
)

KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

##### Initialize Embedding Model

In [4]:
fastembed = TextEmbedding()

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

### Data Preparation


##### Download Dataset
We'll use the Paul Graham Essay Dataset as our corpus.

In [5]:
!llamaindex-cli download-llamadataset PaulGrahamEssayDataset --download-dir ./data

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  4.37it/s]
Successfully downloaded PaulGrahamEssayDataset to ./data


### Create a KDB.AI session and table

In [ ]:
KDBAI_TABLE_NAME = "paul_graham"
session = kdbai.Session(endpoint=KDBAI_ENDPOINT, api_key=KDBAI_API_KEY)
database = session.database("default")

# Drop existing table if it exists
try:
    database.table(KDBAI_TABLE_NAME).drop()
except kdbai.KDBAIException:
    pass

In [7]:
# Define table schema

schema = [
    dict(name="text", type="bytes"),
    dict(name="embedding", type="float32s")
]
index_name = "flat_index"
indexes = [dict(name=index_name, column="embedding", type="flat", params=dict(metric="L2", dims=384))]

table = database.create_table(KDBAI_TABLE_NAME, schema=schema, indexes=indexes)

#### Load and Parse Documents

In [8]:
node_parser = SentenceSplitter(chunk_size=500, chunk_overlap=100)
essays = SimpleDirectoryReader(input_dir="./data/source_files").load_data()
docs = node_parser.get_nodes_from_documents(essays)
len(docs)

46

##### Generate Embeddings

In [ ]:
embedding_model = TextEmbedding()
documents = [doc.text for doc in docs]
embeddings = list(embedding_model.embed(documents))

In [10]:
print(documents[1])

So I'm not surprised I can't remember any programs I wrote, because they can't have done much. My clearest memory is of the moment I learned it was possible for programs not to terminate, when one of mine didn't. On a machine without time-sharing, this was a social as well as a technical error, as the data center manager's expression made clear.

With microcomputers, everything changed. Now you could have a computer sitting right in front of you, on a desk, that could respond to your keystrokes as it was running instead of just churning through a stack of punch cards and then stopping. [1]

The first of my friends to get a microcomputer built it himself. It was sold as a kit by Heathkit. I remember vividly how impressed and envious I felt watching him sitting in front of it, typing programs right into the computer.

Computers were expensive in those days and it took me years of nagging before I convinced my father to buy one, a TRS-80, in about 1980. The gold standard then was the Appl

#####  Insert Data into KDB.AI Table

In [11]:
records_to_insert_with_embeddings = pd.DataFrame({
    "text": [d.encode('utf-8') for d in documents],
    "embedding": embeddings
})

In [12]:
table.insert(records_to_insert_with_embeddings)

{'rowsInserted': 46}

### RAG Implementation

##### Define Query and Generate Embedding

In [13]:
query = "How does Paul Graham decide what to work on?"

In [14]:
query_embedding = list(embedding_model.embed([query]))[0].tolist()

##### Perform Vector Search

In [15]:
search_results = table.search({index_name: [query_embedding]}, n=10)
search_results_df = search_results[0]

In [16]:
pd.set_option('display.max_colwidth', None)
print("Top Search Results Based on Query:", query)
df = pd.DataFrame(search_results_df)
df.head(2)

Top Search Results Based on Query: How does Paul Graham decide what to work on?


,__nn_distance,text,embedding
0,0.823007,"b'In late 2015 I spent 3 months writing essays, and when I went back to working on Bel I could barely understand the code. Not so much because it was badly written as because the problem is so convoluted. When you\'re working on an interpreter written in itself, it\'s hard to keep track of what\'s happening at what level, and errors can be practically encrypted by the time you get them.\n\nSo I said no more essays till Bel was done. But I told few people about Bel while I was working on it. So for years it must have seemed that I was doing nothing, when in fact I was working harder than I\'d ever worked on anything. Occasionally after wrestling for hours with some gruesome bug I\'d check Twitter or HN and see someone asking ""Does Paul Graham still code?""\n\nWorking on Bel was hard but satisfying. I worked on it so intensively that at any given time I had a decent chunk of the code in my head and could write more there. I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with some problem involving continuations while I watched them play in the tide pools. It felt like I was doing life right. I remember that because I was slightly dismayed at how novel it felt. The good news is that I had more moments like this over the next few years.\n\nIn the summer of 2016 we moved to England. We wanted our kids to see what it was like living in another country, and since I was a British citizen by birth, that seemed the obvious choice. We only meant to stay for a year, but we liked it so much that we still live there. So most of Bel was written in England.\n\nIn the fall of 2019, Bel was finally finished. Like McCarthy\'s original Lisp, it\'s a spec rather than an implementation, although like McCarthy\'s Lisp it\'s a spec expressed as code.\n\nNow that I could write essays again, I wrote a bunch about topics I\'d had stacked up. I kept writing essays through 2020, but I also started to think about other things I could work on. How should I choose what to do?'","[-0.05267877, 0.005840427, -0.01187801, -0.028083289, 0.029767925, -0.01268333, -0.009753024, -0.011209541, 0.030792488, -0.07470311, 0.0005716741, 0.034681723, -0.0025648128, -0.007870674, -0.037071493, -0.0026503617, -0.030294443, -0.046712548, -0.026220752, -0.010382689, -0.047210008, 0.0039388337, -0.009324926, 0.04539282, 0.04298206, 0.051068194, 0.029527958, -0.012021941, -0.051774003, -0.20419116, -0.019487105, 0.03856181, 0.054865412, -0.024023462, 0.005628216, 0.059498444, -0.023029648, -0.011461271, 0.0007990732, 0.01532533, 0.013435846, 0.009714834, 0.010104686, -0.014338494, 0.004052569, 0.020879505, 0.0112869395, -0.048422333, 0.025670612, 0.033183247, -0.071020156, -0.032056253, -0.0013147242, 0.045764726, -0.023884403, 0.013609344, 0.021824384, 0.0791942, 0.0021155155, -0.0058458406, 0.022163069, -0.0010415328, -0.1377265, 0.05194325, -0.035091735, 0.020503322, -0.03358411, -0.039575316, -0.018544003, 0.07090187, -0.030203853, 0.0024145627, -0.050365325, 0.1062729, 0.04504893, 0.020158818, -0.0055481945, 0.0020900085, 0.014658697, -0.01600323, 0.018643875, -0.020128626, 0.001960821, 0.014573526, -0.018745624, -0.011082115, -0.026627902, 0.035287272, 0.033186108, 0.004842385, 0.04288919, -0.051519115, 0.021143924, 0.03511711, -0.032461487, -0.053802498, -2.9269107e-05, 0.022274038, -0.019326271, 0.5066904, ...]"
1,0.851789,"b""He wanted to start a startup to make nuclear reactors. But I kept at it, and in October 2013 he finally agreed. We decided he'd take over starting with the winter 2014 batch. For the rest of 2013 I left running YC more and more to Sam, partly so he could learn the job, and partly because I was focused on my mother, whose cancer had returned.\n\nShe died on January 15, 2014. We knew this was coming, but it was still hard when it did.\n\nI kept working on YC till March, to help get that batch of startups through Demo Day, then I checked out pretty completely. (I st

##### RAG Function Definition

In [17]:
def RAG(retrieved_data,prompt):
    messages = "Answer the following query in three sentences based on the context and only the context: " + "\n"
    messages += prompt + "\n"
    if len(retrieved_data) > 0:
        messages += "Context: " + "\n"
        for data in retrieved_data:
            messages += data.decode('utf-8') + "\n"
    openai.api_key = os.environ["OPENAI_API_KEY"]
    response = openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {
            "role": "user",
            "content": [
                {"type": "text", "text": messages},
            ],
            },
        ],
        max_tokens=300,
    )
    content = response.choices[0].message.content
    return content

##### Execute RAG Pipeline

In [18]:
# Utility Function for Text Wrapping

def print_wrapped(text, width=80):
    wrapper = textwrap.TextWrapper(width=width)
    word_list = wrapper.wrap(text=text)
    for line in word_list:
        print(line)

In [19]:
print("Query:", query)

print_wrapped(RAG(search_results_df["text"],query))

Query: How does Paul Graham decide what to work on?
Paul Graham decides what to work on based on a mix of personal interest, the
desire to explore untapped potential in various fields, and the influence of
pivotal moments and advice from close acquaintances. His transition from working
on Y Combinator, painting, and writing essays to developing the Bel programming
language and exploring startup ideas, such as the web app for creating web apps,
reflects a combination of seeking out deeply engaging projects and responding to
unsolicited advice from trusted collaborators that prompts reflection on his
trajectory. Graham's choices are driven by the pursuit of projects that not only
challenge him but also promise a significant impact or learning opportunity,
reflecting a deliberate process of selection influenced by both internal
motivations and external inputs.


### Drop Table To Conserve Resources

In [20]:
table.drop()